In [1]:
import os 
import re
from langchain_community.llms import Ollama
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

In [6]:
# Modified following https://www.youtube.com/watch?v=02cdCd43Ccc 
API_KEY = "NA" #Causes ChatOpenAI to look for local models. base-url is needed to use ollama
Model = 'llama3'
gpt_llm = ChatOpenAI(api_key = API_KEY,model= Model, base_url="http://localhost:11434/v1")
gpt_llm.invoke('what is a bot')

AIMessage(content='A bot (short for "robot") is a software program that runs automatically and interacts with humans or other programs through text-based interfaces, such as:\n\n1. **Chatbots**: Bots that chat with people on platforms like messaging apps, social media, or websites. They respond to inputs and provide information, perform tasks, or even entertain.\n2. **Bots for customer service**: Bots designed to help companies manage customer inquiries and support requests, freeing up human representatives for more complex issues.\n3. **Social bots**: Bots that automate social media tasks, such as posting updates, responding to comments, or analyzing audience engagement.\n4. **Messaging bots**: Bots that interact with users through messaging apps like WhatsApp, Facebook Messenger, or Slack, helping them perform various tasks.\n\nBots typically use natural language processing (NLP) and machine learning algorithms to understand human input, respond accordingly, and learn from the intera

In [8]:
parser = StrOutputParser()
gpt_chain = gpt_llm|parser
gpt_chain.invoke('what is a shark')

'A question that sparks excitement and curiosity in many!\n\nSharks are a type of fish that belong to the class Chondrichthyes. They are characterized by a cartilaginous skeleton (meaning their skeleton is made up of cartilage, not bone), a streamlined body shape, and a unique set of adaptations that make them formidable predators in the ocean.\n\nHere are some fascinating facts about sharks:\n\n1. **Diversity**: There are over 500 species of sharks, ranging from tiny dwarf lanternsharks to massive whale sharks.\n2. **Evolutionary history**: Sharks have been on Earth for around 400 million years, long before humans or even dinosaurs existed!\n3. **Body shape**: Most sharks have a torpedo-shaped body that allows them to swim quickly and efficiently. Some species, like the hammerhead shark, have a unique head shape that provides greater maneuverability.\n4. **Dentition**: Sharks have rows of sharp, tooth-like structures called denticles on their snouts or jaws. These are used for catchin

In [9]:
loader = TextLoader('data.txt',encoding = 'utf-8')
document = loader.load()

In [10]:
spliter = RecursiveCharacterTextSplitter(chunk_size = 200,chunk_overlap = 50)
chunks = spliter.split_documents(document)
chunks[3]


Document(page_content='The roughly 10-centimetre-long cube is made of magnolia-wood panels and has an aluminium frame, solar panels, circuit boards and sensors. The panels incorporate Japanese wood-joinery methods that do', metadata={'source': 'data.txt'})

In [21]:
# vector_storage = FAISS.from_documents(chunks,OpenAIEmbeddings())
# retriever = vector_storage.as_retriever()

from langchain_community.embeddings import OllamaEmbeddings

ollama_emb = OllamaEmbeddings(base_url = 'http://localhost:11434', model="llama3")
# May need: 
# !pip install faiss-cpu
# or 
# !pip install faiss-gpu

vector_storage = FAISS.from_documents(chunks, ollama_emb)
retriever = vector_storage.as_retriever()

In [22]:
retriever.invoke('what is the main characteristic of the satellite')

[Document(page_content='LignoSat will cost about US$191,000 to design, manufacture, launch and operate. Sensors onboard will evaluate strain on the wood, temperature, geomagnetic forces and cosmic radiation, as well as', metadata={'source': 'data.txt'}),
 Document(page_content='Timber pioneers', metadata={'source': 'data.txt'}),
 Document(page_content='And there is a precedent for spacecraft with wooden parts. Launched in 1962, NASA’s Ranger 3 lunar probe had a balsa-wood casing intended to protect its capsule as it landed on the lunar surface (the', metadata={'source': 'data.txt'}),
 Document(page_content='But Murata says that the team has studied measurements of GCR and solar energetic particles — high-energy particles that are released from the Sun — taken by NASA’s Curiosity rover on Mars, as well', metadata={'source': 'data.txt'})]

In [23]:
template = ("""
You are AI-powered chatbot designed to provide 
information and assistance for customers
based on the context provided to you only. 
            
Context:{context}
Question:{question}
""")

In [24]:
prompt = PromptTemplate.from_template(template=template)
prompt.format(
    context = ' Here is a context to use',
    question = ' This is a question to answer'
)

'\nYou are AI-powered chatbot designed to provide \ninformation and assistance for customers\nbased on the context provided to you only. \n            \nContext: Here is a context to use\nQuestion: This is a question to answer\n'

In [25]:
result = RunnableParallel(context = retriever,question = RunnablePassthrough())
chain = result |prompt | gpt_llm | parser

In [26]:
chain.invoke('What is the recommended material for satelites? ')

"According to the context provided, the recommended material for satellites appears to be wood. The conversation mentions that Salim notes the need to confirm the structural integrity, safety, and longevity of wood in space. Additionally, it's mentioned that using wood will make the satellite more sustainable and less polluting compared to conventional materials used in satellites. Furthermore, it's stated that wood does not block radio waves, making it suitable for enclosing an antenna. Therefore, based on this context, it appears that wood is the recommended material for satellites."

In [27]:
chain.invoke('are there any precedents to wood satellites? Do they work?')

'Interesting question! Based on the context provided, I found some relevant information.\n\nThere is ongoing research and exploration into using wood in space. While there may not be specific "wood satellites" with precedents, there are experiments and projects that involve using wood or wooden materials in space. \n\nFor instance, according to an article from 2020, researchers are exploring the potential of using wood in space missions as a sustainable option. The project aims to develop structures and materials that can withstand the harsh conditions of space for long periods. \n\nSalim, the expert quoted earlier, notes that the structural integrity, safety, and longevity of wood need to be confirmed in space. He also mentions that Growth Rate (GCR) and its potential degradation of mechanical properties are important factors to consider.\n\nIt\'s not straightforward to say whether these wooden structures or materials have been successfully tested or put into practice yet. The project

In [35]:
# https://python.langchain.com/v0.2/docs/integrations/document_loaders/url/
# !pip install unstructured
from langchain_community.document_loaders import UnstructuredURLLoader

loaderhmlt = UnstructuredURLLoader(urls = ["https://www.nature.com/articles/d41586-024-01456-z"])
data = loaderhmlt.load()
spliterhtml = RecursiveCharacterTextSplitter(chunk_size = 200,chunk_overlap = 50)
chunkhtml = spliterhtml.split_documents(data)
chunkhtml[3]

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jorgepinzon/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Document(page_content='View author publications', metadata={'source': 'https://www.nature.com/articles/d41586-024-01456-z'})

In [93]:
# The article of interest is inclluded in the first 44 chunks 
vector_storagehtml = FAISS.from_documents(chunkhtml[:44], ollama_emb)
retrieverhtml = vector_storagehtml.as_retriever()

In [94]:
resulthtml = RunnableParallel(context = retrieverhtml,question = RunnablePassthrough())
chainhtml = resulthtml |prompt | gpt_llm | parser
chain_htmlr = chainhtml.invoke('are there any precedents to wood satellites? Do they work?')
chain_htmlr

'According to the provided context, it seems that the idea of using wood in space is not a new one. The article mentions that the project began in 2020, which implies that there might be some precedents or existing research on this topic.\n\nHowever, the specific question of whether wood satellites actually work is left unanswered. It\'s possible that the answer may depend on factors such as the type of wood used, its durability and resistance to radiation in space, and any necessary modifications for use in a space environment.\n\nWhat we do know is that Salim notes that "radiation on Mars is a big problem" and that wood might not be affected by this issue. Additionally, there is mention of confirming the structural integrity, safety, and longevity of wood in space, which implies that some challenges or uncertainties may still exist.\n\nOverall, while there might be some precedents for using wood in space, it appears that more research and testing would be necessary to confirm its eff

In [95]:
chain_htmlr2 = chainhtml.invoke('What is the recommended material for satelites? ')
chain_htmlr2

"Based on the context you provided, it appears that wood is being considered as a potential material for satellites. According to the document, Salim noted that wood's structural integrity, safety and longevity need to be confirmed in space. Additionally, the documents suggest that wood could be a sustainable and less polluting option compared to conventional metals used in satellite construction.\n\nIt seems that wood's properties, such as being resilient in harsh environments like space, not blocking radio waves, make it a suitable material for encasing an antenna. The documents also mention that using wood in satellites might be explored, likely due to its potential benefits in the aerospace industry.\n\nTherefore, based on this context, I would recommend considering wood as a material for satellite construction, pending further confirmation of its structural integrity and safety in space."

In [96]:
chain_htmlr2 = chainhtml.invoke('What is the recommended material for satelites? ')
chain_htmlr2

'Based on the provided context, it seems that the recommended material for satellites is wood. According to the documents, wood is seen as a potential option due to its structural integrity, safety, and longevity in space, making it suitable for enclosing an antenna without blocking radio waves. Additionally, using wood can lead to more sustainable and less polluting satellites compared to conventional satellites made of metals.'

In [97]:
chain_htmlr2 = re.sub(r'\. ',r'.\n', chain_htmlr2)
print(chain_htmlr2)

Based on the provided context, it seems that the recommended material for satellites is wood.
According to the documents, wood is seen as a potential option due to its structural integrity, safety, and longevity in space, making it suitable for enclosing an antenna without blocking radio waves.
Additionally, using wood can lead to more sustainable and less polluting satellites compared to conventional satellites made of metals.


In [98]:
chain_htmlr3 = chainhtml.invoke('What is the main idea of the context provided? ')
chain_htmlr3
chain_htmlr3 = re.sub(r'\. ',r'.\n', chain_htmlr3)
print(chain_htmlr3)

Based on the context provided, the main idea seems to be about exploring the use of Japanese wood-joinery methods in space-based applications.
The text mentions Salim's comment about the structural integrity and longevity of wood, as well as the need to confirm these factors in space.
There is also a reference to the environmental effects of pollution being unknown.
Overall, the main idea appears to be discussing innovative uses for Japanese wood-joinery techniques in aerospace-related applications.
